In [1]:
import os

import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader, Subset, ConcatDataset
import numpy as np

from tqdm import tqdm
import random

In [2]:
import model

In [3]:
ENCODER = 'densenet161'
ENCODER_WEIGHTS = 'imagenet'

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [4]:
unet = smp.Unet(encoder_name=ENCODER, 
                 encoder_weights=ENCODER_WEIGHTS,
                decoder_attention_type=None,
                 in_channels=3, classes=1, activation="sigmoid", aux_params=None)
optimizer = "sgd"

In [5]:
encoder = unet.encoder.features[:9]
encoder

Sequential(
  (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(144, 192, 

In [6]:
decoder = model.SCGDecoder(None, None)
decoder

SCGDecoder(
  (encoder): SCG(
    (pooling): AdaptiveAvgPool2d(output_size=(32, 32))
    (mu): Sequential(
      (0): Conv2d(2112, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Dropout(p=0.2, inplace=False)
    )
    (log_var): Sequential(
      (0): Conv2d(2112, 1, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
      (1): Dropout(p=0.2, inplace=False)
    )
  )
  (decoder): Sequential(
    (0): GCN_Layer(
      (fc): Sequential(
        (0): Linear(in_features=2112, out_features=512, bias=True)
        (1): BatchNorm_GCN(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (1): GCN_Layer(
      (fc): Sequential(
        (0): Linear(in_features=512, out_features=1, bias=True)
      )
    )
  )
)

In [7]:
scg_net = model.SCGNet(encoder=encoder, 
               decoder=decoder,)